In [10]:
from collections import defaultdict
from pathlib import Path

import gurobipy as gp
import polars as pl
from gurobipy import GRB


In [11]:



def get_data(data_dir: str, discount_factor: int = 5):
    df = pl.read_csv(
        Path(f"{data_dir}/alternatives.csv"), schema_overrides={"unit": pl.Int64}, infer_schema_length=1000
    )
    df = df.select(
        [
            "unit",
            "schedule",
            f"npv_{discount_factor}_percent",
            "stock_0",
            "stock_5",
            "stock_25",
            "harvest_value_5",
            "harvest_value_25",
            "stock_1_0",
            "stock_1_5",
            "stock_1_25",
            "stock_2_0",
            "stock_2_5",
            "stock_2_25",
            "stock_30_0",
            "stock_30_5",
            "stock_30_25",
        ]
    )
    df_keys = pl.read_csv(
        Path(f"{data_dir}/alternatives_key.csv"), schema_overrides={"unit": pl.Int64}, infer_schema_length=1000
    )
    df_keys = df_keys.drop("holding")
    # Split the treatments into multiple columns, to make them easier to read or whatever
    df_keys = df_keys.with_columns(
        [
            pl.when(pl.col("treatment").str.contains("_5"))
            .then(pl.col("treatment").str.extract(r"(\w+)_5", 1))
            .otherwise(pl.lit("donothing"))
            .alias("treatment_5"),
            pl.when(pl.col("treatment").str.contains("_25"))
            .then(pl.col("treatment").str.extract(r"(\w+)_25", 1))
            .otherwise(pl.lit("donothing"))
            .alias("treatment_25"),
        ]
    )

    df = df_keys.join(df, on=["unit", "schedule"], how="inner")
    return df

In [ ]:
model = gp.Model()

discount_factor = 5
data_dirs = ["C:/MyTemp/data/two_stage_data/h1h23", "C:/MyTemp/data/two_stage_data/h1h23"]
n_scenarios = len(data_dirs)

alpha = model.addVar()
normalsum = 0
hsum5=[]
hsum25=[]
npvsum=[]
stocksum=[]
mvars=[]
unitsums=[]
harvest5 = []
harvest25 = []
stock = []
npv = []
x0_vars = {}
x0_unit_sum = defaultdict(float)

for i in range(n_scenarios):
    hsum5.append(0)
    hsum25.append(0)
    npvsum.append(0)
    stocksum.append(0)
    mvars.append({})
    unitsums.append(defaultdict(float))

    df = get_data(data_dirs[i], discount_factor=discount_factor)

    for row in df.iter_rows(named=True):
        if (row["unit"], row["treatment_5"]) not in x0_vars:
            x0_vars[(row["unit"], row["treatment_5"])] = model.addVar(vtype=GRB.BINARY)
            x0_unit_sum[row["unit"]] += x0_vars[(row["unit"], row["treatment_5"])]
        new_var = model.addVar(vtype=GRB.BINARY)
        mvars[i][(row["unit"], row["treatment_5"], row["treatment_25"])] = new_var
        hsum5[i] += new_var * row["harvest_value_5"]
        hsum25[i] += new_var * row["harvest_value_25"]
        npvsum[i] += new_var * row[f"npv_{discount_factor}_percent"]
        stocksum[i] += new_var * row["stock_25"]
        unitsums[i][(row["unit"], row["treatment_5"])] += new_var

    for key in unitsums[i]:
        model.addConstr(unitsums[i][key] == x0_vars[key])

    harvest5.append(model.addVar())
    model.addConstr(harvest5[i] <= hsum5[i])
    harvest25.append(model.addVar())
    model.addConstr(harvest25[i] <= hsum25[i])
    stock.append(model.addVar())
    model.addConstr(stock[i] <= stocksum[i])
    npv.append(model.addVar())
    model.addConstr(npv[i] <= npvsum[i])


    objectives = {"harvest5": harvest5[i], "harvest25": harvest25[i], "stock": stock[i], "npv": npv[i]}
    ideals = {
        "harvest5": 104856.84974800126,
        "harvest25": 548886.5857012663,
        "stock": 6382.8502845284875,
        "npv": 368608.7206502493,
    }
    nadirs = {"harvest5": 0, "harvest25": 0, "stock": 323.97015986373606, "npv": 174254.640821209}

    reference = {
        "harvest5": 104856.84974800126,
        "harvest25": 548886.5857012663,
        "stock": 6382.8502845284875,
        "npv": 368608.7206502493,
    }


    for obj in objectives:
        model.addConstr(alpha >= (objectives[obj] - reference[obj]) / (nadirs[obj] - ideals[obj]))
        normalsum += objectives[obj] / (nadirs[obj] - ideals[obj])

for unitsum in x0_unit_sum.values():
    model.addConstr(unitsum == 1)

rho = 1e-6
model.setObjective(alpha + rho * normalsum, sense=GRB.MINIMIZE)
model.optimize()

print(model.ObjVal)
print(harvest5[0].x)
print(harvest5[1].x)
print(harvest25[0].x)
print(harvest25[1].x)
print(stock[0].x)
print(stock[1].x)
print(npv[0].x)
print(npv[1].x)



Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-1245U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 60 rows, 469 columns and 2014 nonzeros
Model fingerprint: 0xaf19a9af
Variable types: 9 continuous, 460 integer (460 binary)
Coefficient statistics:
  Matrix range     [2e-06, 1e+05]
  Objective range  [2e-12, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 1.8965825
Presolve removed 32 rows and 260 columns
Presolve time: 0.00s
Presolved: 28 rows, 209 columns, 852 nonzeros
Found heuristic solution: objective 1.7860045
Variable types: 5 continuous, 204 integer (204 binary)

Root relaxation: objective 5.376495e-01, 73 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf 